## ISM 6251-003

## Surya Teja Pidakala - U99286608

# Average Price Prediction 

#### We have a set of 9 Predictors that can help us find if the price of a certain item is above the average price or less than the average price. Those Predictors are 'YEAR', 'MONTH', 'DAY', 'ORDER', 'COUNTRY', 'LOCATION', 'PRICE', 'AVG_PRICE', 'PAGE'

I have defined the metrics of a Confusion Matrix and made some assumptions on how much will be the net loss to get a clear idea on which metric is critical.

Before anything, let us discuss the Confusion Matrix in detail.

* True Positive  - This is the case when we predict the price is less than the average price and it actually is true. This a very perfect situation for us because we were right in assessing the situation. So, from a price point, We would have some $10,000

* True Negative  - This is the case when we predict the price of a certain item is more than the average price and it is actually true. This is a case where when we spend nothing and it costs us nothing techinically $0

* False Positive - This is a case when we predict the price of a certain item is less than average price but is more than the average price value. Here, we are wrong in assesing the situation and we might get away with it a couple of times but cannot continue doing this. Let us put this are $100.

* False Negative - This is the case when we predict the price is more than average price but in actuality the price of that particlar item is less than average price. This a very bad failure for us because this will make us loose out a potential customer. So let us put this at $50,000.

Finally, we can see that False Negatives are something that we should avoid as much as we can

# 1.0 Let us import all the necessary libraries first!

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score ,f1_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### 1.1 Let us now load the data into a variable

In [19]:
df = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/Online_website.csv")
df.head(5)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,Avg Price,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,0,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,0,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,0,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


Let us know the type of data that we are dealing with

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2046 entries, 0 to 2045
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   year                     2046 non-null   int64 
 1   month                    2046 non-null   int64 
 2   day                      2046 non-null   int64 
 3   order                    2046 non-null   int64 
 4   country                  2046 non-null   int64 
 5   session ID               2046 non-null   int64 
 6   page 1 (main category)   2046 non-null   int64 
 7   page 2 (clothing model)  2046 non-null   object
 8   colour                   2046 non-null   int64 
 9   location                 2046 non-null   int64 
 10  model photography        2046 non-null   int64 
 11  price                    2046 non-null   int64 
 12  Avg Price                2046 non-null   int64 
 13  page                     2046 non-null   int64 
dtypes: int64(13), object(1)
memory usage: 22

#### 1.2 To make the data clean, let us convert all words to upper case and replace space with an "_"

In [21]:
df.columns = [s.strip().upper().replace(' ', '_') for s in df.columns] 

In [22]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'ORDER', 'COUNTRY', 'SESSION_ID',
       'PAGE_1_(MAIN_CATEGORY)', 'PAGE_2_(CLOTHING_MODEL)', 'COLOUR',
       'LOCATION', 'MODEL_PHOTOGRAPHY', 'PRICE', 'AVG_PRICE', 'PAGE'],
      dtype='object')

In [23]:
df.head()

,YEAR,MONTH,DAY,ORDER,COUNTRY,SESSION_ID,PAGE_1_(MAIN_CATEGORY),PAGE_2_(CLOTHING_MODEL),COLOUR,LOCATION,MODEL_PHOTOGRAPHY,PRICE,AVG_PRICE,PAGE
0,2008,4,1,1,29,1,1,A13,1,5,1,28,0,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,0,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,0,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


#### 1.3 Now, let us see what are the columns that do not contribute towards influencing the target variable. Such columns are of no use to include in our analysis. So, let us go ahead and drop them.

In [24]:
df = df.drop(columns= ['PAGE_1_(MAIN_CATEGORY)', 'PAGE_2_(CLOTHING_MODEL)', 'COLOUR', 'MODEL_PHOTOGRAPHY','SESSION_ID'])

In [25]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'ORDER', 'COUNTRY', 'LOCATION', 'PRICE',
       'AVG_PRICE', 'PAGE'],
      dtype='object')

Let us check for null values in the data after dropping the columns.

In [26]:
df[df.isnull().any(axis=1)]

,YEAR,MONTH,DAY,ORDER,COUNTRY,LOCATION,PRICE,AVG_PRICE,PAGE


## 2.0 Predictors and Target Variable split

### 2.1 Partitioning data into training and test data sets


In [27]:
df_train, df_test = train_test_split(df, test_size=0.3)

## 2.2 Let us now divide the set of variables into Target and Predictor classes.

In [28]:
target = 'AVG_PRICE'
predictors = list(df.columns)
predictors.remove(target)

In [29]:
train_X = df_train[predictors]
test_X = df_test[predictors]
train_y = df_train[target]
test_y = df_test[target]

Let us know some key aspects with regards to the balance of data.

## 2.3 Let us use a Random OverSampler to resize the data

In [30]:
train_y.value_counts()

0    730
1    702
Name: AVG_PRICE, dtype: int64

We can see that there is an slight imbalance in the data, So lets address this issue

In [31]:
ros = RandomOverSampler(random_state=1)
train_X, train_y = ros.fit_resample(train_X, train_y)
train_y.value_counts()

0    730
1    730
Name: AVG_PRICE, dtype: int64

## 2.4 Now, as we addressed the data imbalance issue, we can go ahead with the analysis of the data

## 2.5 Let us now save this data into individual CSV files

In [32]:
df_train.to_csv('./data/online_price_train_df.csv', index=False)
train_X.to_csv('./data/online_price_train_X.csv', index=False)
train_y.to_csv('./data/online_price_train_y.csv', index=False)
df_test.to_csv('./data/online_price_test_df.csv', index=False)
test_X.to_csv('./data/online_price_test_X.csv', index=False)
test_y.to_csv('./data/online_price_test_y.csv', index=False)